In [1]:
import requests
import pandas as pd
from lxml import html
from datetime import datetime

In [2]:
communities = ['Omaha', 'Lincoln', 'Norfolk', 'Fremont']
html_string = 'http://results.bigredkeno.com/?community=INPUT_COMM'

In [3]:
# Function that extracts numbers from a list, 20 at a time:
def extract_values(input_list):
    copied_list = input_list.copy()
    final_df = pd.DataFrame()
    while len(copied_list) > 0:
        temp_df = pd.DataFrame(copied_list[:20])
        temp_df = temp_df.T
        if final_df.shape[0] == 0:
            final_df = temp_df.copy()
        else:
            final_df = final_df.append(temp_df)
        copied_list = copied_list[20:]
    return final_df

In [4]:
for each_community in communities:
    temp_string = html_string.replace('INPUT_COMM', each_community)
    current_request = requests.get(temp_string)
    tree = html.fromstring(current_request.content)
    # Getting the numbers:
    raw_number_lists = tree.xpath('//td')
    # Extracting the numbers:
    extracted_numbers = [int(item.text) for item in raw_number_lists if item.text != None and item.text.strip() != '']
    # Converting the extracted numbers to a table:
    table_of_data = extract_values(extracted_numbers)
    # Opening the previous data file:
    previous_data = pd.read_excel('./data/')
    table_of_data.to_excel('./data/%s.xlsx' % each_community.lower())